- 在这个任务中，你将构建 Transformer 架构中的编码器部分。
- 将使用 PyTorch 框架实现以下组件
  - 多头注意力（MHA）模块
  - 位置感知前馈神经网络
  - 输出层接收编码器输出并预测 token_ids。
  - (可选地)研究添加位置信息是否有帮助。

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import math

In [ ]:
# 位置编码
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)


In [22]:
max_len=6
d_model = 4

In [23]:
pe = torch.zeros(max_len, d_model)
pe

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [24]:
position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
position

tensor([[0.],
        [1.],
        [2.],
        [3.],
        [4.],
        [5.]])

In [27]:
div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
div_term

tensor([1.0000, 0.0100])

In [30]:
pe[:, 0::2]

tensor([[ 0.0000,  0.0000],
        [ 0.8415,  0.0100],
        [ 0.9093,  0.0200],
        [ 0.1411,  0.0300],
        [-0.7568,  0.0400],
        [-0.9589,  0.0500]])

In [29]:
pe[:, 0::2] = torch.sin(position * div_term)
pe

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.8415,  0.0000,  0.0100,  0.0000],
        [ 0.9093,  0.0000,  0.0200,  0.0000],
        [ 0.1411,  0.0000,  0.0300,  0.0000],
        [-0.7568,  0.0000,  0.0400,  0.0000],
        [-0.9589,  0.0000,  0.0500,  0.0000]])

In [32]:
pe[:, 1::2] = torch.cos(position * div_term)
pe

tensor([[ 0.0000,  1.0000,  0.0000,  1.0000],
        [ 0.8415,  0.5403,  0.0100,  0.9999],
        [ 0.9093, -0.4161,  0.0200,  0.9998],
        [ 0.1411, -0.9900,  0.0300,  0.9996],
        [-0.7568, -0.6536,  0.0400,  0.9992],
        [-0.9589,  0.2837,  0.0500,  0.9988]])

In [34]:
pe = pe.unsqueeze(0).transpose(0, 1)
pe

tensor([[[[ 0.0000,  1.0000,  0.0000,  1.0000]]],


        [[[ 0.8415,  0.5403,  0.0100,  0.9999]]],


        [[[ 0.9093, -0.4161,  0.0200,  0.9998]]],


        [[[ 0.1411, -0.9900,  0.0300,  0.9996]]],


        [[[-0.7568, -0.6536,  0.0400,  0.9992]]],


        [[[-0.9589,  0.2837,  0.0500,  0.9988]]]])

$$
PE_{(pos,2i)} = \sin\left(\frac{pos}{10000^{2i/d_{\text{model}}}}\right),
$$

$$
PE_{(pos,2i+1)} = \cos\left(\frac{pos}{10000^{2i/d_{\text{model}}}}\right).
$$

其中，pos表示位置，d_model表示模型的维度，一般设置为512。


`PositionalEncoding` 类是 Transformer 模型中一个非常重要的组件，它的作用是为输入的嵌入向量（embedding）添加位置信息。Transformer 是一个基于序列的模型，但它本身并不像 RNN 那样能够自然地处理序列中的位置信息。因此，需要通过某种方式将位置信息注入到模型中，`PositionalEncoding` 就是实现这一功能的关键部分。

### 代码解析

#### 1. 初始化方法 `__init__`
```python
def __init__(self, d_model, dropout=0.1, max_len=5000):
    super(PositionalEncoding, self).__init__()
    self.dropout = nn.Dropout(p=dropout)
    pe = torch.zeros(max_len, d_model)
    position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)
    pe = pe.unsqueeze(0).transpose(0, 1)
    self.register_buffer('pe', pe)
```

- **`d_model`**：表示嵌入向量的维度，即每个位置编码的大小。
- **`dropout`**：用于正则化，防止过拟合。
- **`max_len`**：表示模型能够处理的最大序列长度。

#### 生成位置编码
位置编码的生成基于论文《Attention Is All You Need》中提出的方法。具体来说，位置编码是一个固定大小的向量（维度为 `d_model`），它通过正弦和余弦函数生成。这种编码方式能够捕捉到位置信息，并且可以处理比训练时序列长度更长的序列。

- **`torch.zeros(max_len, d_model)`**：创建一个形状为 `(max_len, d_model)` 的零矩阵，用于存储位置编码。
- **`torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)`**：生成一个从 0 到 `max_len-1` 的序列，并将其转换为浮点数，然后通过 `unsqueeze(1)` 将其扩展为二维张量，形状为 `(max_len, 1)`。这表示每个位置的索引。
- **`torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))`**：计算分母部分，用于调整正弦和余弦函数的频率。`torch.arange(0, d_model, 2)` 表示每隔一个维度取一个值（因为 `d_model` 是偶数），`-math.log(10000.0) / d_model` 是一个缩放因子，用于控制频率的变化范围。
- **`pe[:, 0::2] = torch.sin(position * div_term)`** 和 **`pe[:, 1::2] = torch.cos(position * div_term)`**：分别将正弦和余弦函数的值赋值到位置编码矩阵的偶数和奇数位置上。这样，每个位置编码的偶数维度由正弦函数生成，奇数维度由余弦函数生成。
- **`pe = pe.unsqueeze(0).transpose(0, 1)`**：将位置编码矩阵的形状调整为 `(max_len, 1, d_model)`，然后转置为 `(1, max_len, d_model)`，以便在后续操作中可以直接与输入张量相加。
- **`self.register_buffer('pe', pe)`**：将位置编码矩阵注册为一个缓冲区（buffer），这样它不会被视为模型的参数，但会在模型的 `state_dict` 中保存，并且会随着模型的移动（如从 CPU 移动到 GPU）而自动移动。
在代码中，位置编码（Positional Encoding）的生成是基于论文《Attention Is All You Need》中提出的公式。具体公式如下：

对于每个位置 \( pos \) 和每个维度 \( i \)，位置编码 \( PE(pos, i) \) 的计算公式为：

\[
PE(pos, i) = 
\begin{cases}  
\sin\left(\frac{pos}{10000^{2i/d_{model}}}\right) & \text{if } i \text{ is even} \\  
\cos\left(\frac{pos}{10000^{2i/d_{model}}}\right) & \text{if } i \text{ is odd}  
\end{cases}
\]

其中：
- \( pos \) 是位置索引（从 0 开始）。
- \( i \) 是维度索引（从 0 开始）。
- \( d_{model} \) 是嵌入向量的维度。
- \( 10000 \) 是一个常数，用于控制频率的变化范围。

### 代码与公式对应关系

在代码中，位置编码的生成过程与上述公式一一对应：

```python
pe = torch.zeros(max_len, d_model)
position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
pe[:, 0::2] = torch.sin(position * div_term)
pe[:, 1::2] = torch.cos(position * div_term)
```

1. **`position`**：
   ```python
   position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
   ```
   这里生成了一个从 0 到 `max_len-1` 的序列，表示每个位置的索引 $pos$。`unsqueeze(1)` 将其扩展为二维张量，形状为 `(max_len, 1)`，方便后续的广播操作。

2. **`div_term`**：
   ```python
   div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
   ```
   这里计算了公式中的分母部分：
   \[
   \frac{1}{10000^{2i/d_{model}}}
   \]
   其中：
   - `torch.arange(0, d_model, 2)` 表示每隔一个维度取一个值（因为偶数维度用正弦，奇数维度用余弦）。
   - `(-math.log(10000.0) / d_model)` 是公式中的指数部分。
   - `torch.exp(...)` 计算 \( e^{x} \)，从而得到分母的值。

3. **`pe[:, 0::2]` 和 `pe[:, 1::2]`**：
   ```python
   pe[:, 0::2] = torch.sin(position * div_term)
   pe[:, 1::2] = torch.cos(position * div_term)
   ```
   这里分别计算了偶数维度和奇数维度的位置编码：
   - `pe[:, 0::2]` 对应公式中的正弦部分：
     $$
     PE(pos, i) = \sin\left(\frac{pos}{10000^{2i/d_{model}}}\right) \quad \text{(for even } i\text{)}
     $$
   - `pe[:, 1::2]` 对应公式中的余弦部分：
     $$
     PE(pos, i) = \cos\left(\frac{pos}{10000^{2i/d_{model}}}\right) \quad \text{(for odd } i\text{)}
     $$
### 1.3max_len的选择
在代码中，`max_len` 表示模型能够处理的最大序列长度。这个值是预先设定的，用于定义位置编码矩阵的大小。在实际应用中，`max_len` 应该足够大，以覆盖你预期中可能遇到的最长序列。
原始代码在实际输入时，只使用与输入序列长度相等的部分位置编码
#### 1.3.1 代码中的 `max_len`

```python
pe = torch.zeros(max_len, d_model)
```

这里，`pe` 是一个形状为 `(max_len, d_model)` 的零矩阵，用于存储位置编码。`max_len` 是这个矩阵的行数，表示可以处理的最大序列长度。

#### 1.3.2选择 `max_len` 的值

在实际应用中，选择 `max_len` 的值需要考虑以下因素：

1. **数据集的特性**：如果你的数据集中序列的长度变化很大，那么 `max_len` 应该足够大，以覆盖最长的序列。
2. **模型的容量**：较大的 `max_len` 会增加模型的参数量，因为位置编码矩阵的大小与 `max_len` 直接相关。这可能会增加模型的复杂度和训练时间。
3. **计算资源**：较大的 `max_len` 会占用更多的内存和计算资源，特别是在使用 GPU 进行训练时。

#### 1.3.3 实际应用

在实际应用中，`max_len` 的值通常根据数据集的统计特性来确定。例如，如果数据集中 99% 的序列长度都小于 100，那么可以选择 `max_len = 100`。这样可以确保模型能够处理大多数序列，同时避免不必要的计算开销。

#### 1.3.4总结

`max_len` 是一个重要的超参数，它定义了模型能够处理的最大序列长度。选择合适的 `max_len` 值需要考虑数据集的特性、模型的容量和计算资源。在实际应用中，可以通过分析数据集的统计特性来确定 `max_len` 的值。

#### 2. 前向传播方法 `forward`
```python
def forward(self, x):
    x = x + self.pe[:x.size(0), :]
    return self.dropout(x)
```

- **`x`**：输入的嵌入向量，形状为 `(seq_len, batch_size, d_model)`。
- **`self.pe[:x.size(0), :]`**：根据输入序列的实际长度（`x.size(0)`），从预定义的位置编码矩阵中取出相应长度的位置编码。
- **`x + self.pe[:x.size(0), :]`**：将位置编码加到输入的嵌入向量上。由于位置编码的形状为 `(seq_len, 1, d_model)`，而输入嵌入向量的形状为 `(seq_len, batch_size, d_model)`，PyTorch 会自动进行广播操作，将位置编码应用到每个样本上。
- **`self.dropout(x)`**：对加了位置编码后的嵌入向量应用 Dropout，以防止过拟合。

### 总结
`PositionalEncoding` 类的作用是为输入的嵌入向量添加位置信息，使得 Transformer 模型能够感知序列中每个元素的位置。位置编码通过正弦和余弦函数生成，能够捕捉到位置信息，并且可以处理比训练时序列长度更长的序列。

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(MultiHeadAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert self.head_dim * heads == embed_size, "嵌入尺寸需要被头部整除"

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)

    def forward(self, values, keys, query, mask):
        N = query.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]
        # Split the embedding into self.heads different pieces
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = query.reshape(N, query_len, self.heads, self.head_dim)

        values = self.values(values)
        keys = self.keys(keys)
        queries = self.queries(queries)

        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])

        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        attention = torch.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)
        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )

        out = self.fc_out(out)
        return out

好的！我们再详细解析一下 `MultiHeadAttention` 类的代码，这次我会结合具体的例子来说明每一步的作用和计算过程。

### 1. 初始化方法 `__init__`

```python
def __init__(self, embed_size, heads):
    super(MultiHeadAttention, self).__init__()
    self.embed_size = embed_size
    self.heads = heads
    self.head_dim = embed_size // heads

    assert self.head_dim * heads == embed_size, "Embed size needs to be divisible by heads"

    self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
    self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
    self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
    self.fc_out = nn.Linear(heads * self.head_dim, embed_size)
```

#### 参数解释
- **`embed_size`**：嵌入向量的维度，表示每个输入向量的大小。
- **`heads`**：注意力头的数量。多头注意力机制将输入分割成多个“头”，每个头学习不同的特征。
- **`head_dim`**：每个注意力头的维度大小，计算公式为 `embed_size // heads`。这意味着每个头处理的特征子集的大小。

#### 线性变换层
- **`self.values`**、**`self.keys`**、**`self.queries`**：
  - 这些是线性变换层，用于将输入的嵌入向量分别转换为值（Values）、键（Keys）和查询（Queries）。
  - 每个线性层的输入和输出维度都是 `self.head_dim`，因为每个头处理的特征子集大小为 `self.head_dim`。
  - 使用 `bias=False` 是为了简化计算，避免引入额外的偏置项。

- **`self.fc_out`**：
  - 在多头注意力计算完成后，将所有头的输出拼接起来，并通过一个线性层将维度转换回原始的嵌入维度 `embed_size`。

### 2. 前向传播方法 `forward`

```python
def forward(self, values, keys, query, mask):
    N = query.shape[0]  # Batch size
    value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]
```

#### 输入参数
- **`values`**、**`keys`**、**`query`**：
  - 这三个输入张量的形状通常为 `(batch_size, seq_len, embed_size)`。
  - 它们分别对应于值（Values）、键（Keys）和查询（Queries）。
- **`mask`**：
  - 用于遮蔽某些位置的注意力权重，避免模型关注到不应该关注的部分（例如，解码器中的未来信息）。

#### 多头注意力计算过程

1. **将输入嵌入分割为多个头**：
   ```python
   values = values.reshape(N, value_len, self.heads, self.head_dim)
   keys = keys.reshape(N, key_len, self.heads, self.head_dim)
   queries = query.reshape(N, query_len, self.heads, self.head_dim)
   ```
   - 将输入的嵌入向量分割成 `heads` 个头，每个头的维度为 `self.head_dim`。
   - 例如，如果 `embed_size = 256`，`heads = 8`，则 `self.head_dim = 32`，每个头处理 32 维的特征。
   - 重塑后的形状为 `(N, seq_len, heads, head_dim)`。

2. **线性变换**：
   ```python
   values = self.values(values)
   keys = self.keys(keys)
   queries = self.queries(queries)
   ```
   - 对每个头的值、键和查询分别进行线性变换。
   - 这一步将输入特征投影到不同的子空间中，使得每个头可以学习不同的特征。

3. **计算注意力分数（Attention Scores）**：
   ```python
   energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])
   ```
   - 使用 `torch.einsum` 计算查询和键之间的点积，得到注意力分数矩阵。
   - 公式 `nqhd,nkhd->nhqk` 表示：
     - `n`：批量大小（Batch Size）。
     - `q`：查询序列的长度。
     - `k`：键序列的长度。
     - `h`：头的数量。
     - `d`：每个头的维度。
   - 输出的 `energy` 形状为 `(N, heads, query_len, key_len)`。

4. **应用掩码（Masking）**：
   ```python
   if mask is not None:
       energy = energy.masked_fill(mask == 0, float("-1e20"))
   ```
   - 如果提供了掩码，将掩码为 0 的位置的注意力分数设置为一个非常小的值（如 `-1e20`），这样在后续的 softmax 计算中，这些位置的权重会趋近于 0。

5. **计算注意力权重**：
   ```python
   attention = torch.softmax(energy / (self.embed_size ** (0.5)), dim=3)
   ```
   - 对注意力分数进行 softmax 归一化，得到注意力权重。
   - 除以 `sqrt(embed_size)` 是为了缩放点积结果，避免梯度消失或爆炸。

6. **应用注意力权重**：
   ```python
   out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
       N, query_len, self.heads * self.head_dim
   )
   ```
   - 使用 `torch.einsum` 将注意力权重与值相乘，得到加权的值。
   - 公式 `nhql,nlhd->nqhd` 表示：
     - `n`：批量大小。
     - `h`：头的数量。
     - `q`：查询序列的长度。
     - `l`：值序列的长度。
     - `d`：每个头的维度。
   - 输出的 `out` 形状为 `(N, query_len, heads * self.head_dim)`。

7. **线性变换输出**：
   ```python
   out = self.fc_out(out)
   ```
   - 将所有头的输出拼接起来，并通过一个线性层将维度转换回原始的嵌入维度 `embed_size`。

### 3. 示例矩阵计算

假设：
- `embed_size = 4`
- `heads = 2`
- `head_dim = embed_size // heads = 2`
- 输入序列长度为 3，批量大小为 1。

#### 输入张量
```python
values = torch.tensor([[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]]], dtype=torch.float32)
keys = torch.tensor([[[13, 14, 15, 16], [17, 18, 19, 20], [21, 22, 23, 24]]], dtype=torch.float32)
query = torch.tensor([[[25, 26, 27, 28], [29, 30, 31, 32], [33, 34, 35, 36]]], dtype=torch.float32)
mask = None
```

#### 重塑为多头
```python
values = values.reshape(1, 3, 2, 2)  # (N, value_len, heads, head_dim)
keys = keys.reshape(1, 3, 2, 2)
queries = query.reshape(1, 3, 2, 2)
```

#### 线性变换
假设线性变换层的权重为单位矩阵（简化计算），则：
```python
values = self.values(values)  # 不改变值
keys = self.keys(keys)
queries = self.queries(queries)
```

#### 计算注意力分数
```python
energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])
```

假设：
- `queries = [[[1, 2], [3, 4]], [[5, 6], [7, 8]], [[9, 10], [11, 12]]]`
- `keys = [[[13, 14], [15, 16]], [[17, 18], [19, 20]], [[21, 22], [23, 24]]]`

计算点积：
```python
energy = [
    [
        [[1*13 + 2*14, 1*15 + 2*16], [1*17 + 2*18, 1*19 + 2*20]],


In [40]:
values = torch.tensor([[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]]], dtype=torch.float32)
keys = torch.tensor([[[13, 14, 15, 16], [17, 18, 19, 20], [21, 22, 23, 24]]], dtype=torch.float32)
query = torch.tensor([[[25, 26, 27, 28], [29, 30, 31, 32], [33, 34, 35, 36]]], dtype=torch.float32)
mask = None


In [41]:
values,keys,query

(tensor([[[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.]]]),
 tensor([[[13., 14., 15., 16.],
          [17., 18., 19., 20.],
          [21., 22., 23., 24.]]]),
 tensor([[[25., 26., 27., 28.],
          [29., 30., 31., 32.],
          [33., 34., 35., 36.]]]))

In [38]:
values = values.reshape(1, 3, 2, 2)  # (N, value_len, heads, head_dim)
keys = keys.reshape(1, 3, 2, 2)
queries = query.reshape(1, 3, 2, 2)


In [39]:
values,keys,query

(tensor([[[[ 1.,  2.],
           [ 3.,  4.]],
 
          [[ 5.,  6.],
           [ 7.,  8.]],
 
          [[ 9., 10.],
           [11., 12.]]]]),
 tensor([[[[13., 14.],
           [15., 16.]],
 
          [[17., 18.],
           [19., 20.]],
 
          [[21., 22.],
           [23., 24.]]]]),
 tensor([[[25., 26., 27., 28.],
          [29., 30., 31., 32.],
          [33., 34., 35., 36.]]]))

In [14]:
# Transformer 编码器
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = MultiHeadAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size),
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        attention = self.attention(value, key, query, mask)

        x = self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out

这段代码实现了一个Transformer编码器模块（Transformer Block），它是Transformer架构的核心组件之一。Transformer架构是一种基于自注意力机制（Self-Attention）的深度学习模型，广泛应用于自然语言处理（NLP）任务，如机器翻译、文本生成等。以下是对代码的详细解释：

---

### 1. **类定义**
```python
class TransformerBlock(nn.Module):
```
`TransformerBlock` 是一个继承自 PyTorch 的 `nn.Module` 的类，表示一个Transformer编码器模块。`nn.Module` 是 PyTorch 中所有神经网络模块的基类，用于定义和管理神经网络的结构。

---

### 2. **初始化方法**
```python
def __init__(self, embed_size, heads, dropout, forward_expansion):
    super(TransformerBlock, self).__init__()
    self.attention = MultiHeadAttention(embed_size, heads)
    self.norm1 = nn.LayerNorm(embed_size)
    self.norm2 = nn.LayerNorm(embed_size)

    self.feed_forward = nn.Sequential(
        nn.Linear(embed_size, forward_expansion * embed_size),
        nn.ReLU(),
        nn.Linear(forward_expansion * embed_size, embed_size),
    )

    self.dropout = nn.Dropout(dropout)
```

#### **参数解释**
- `embed_size`: 嵌入向量的维度，表示每个词或标记（token）的特征维度。
- `heads`: 多头注意力机制中的头数（`Multi-Head Attention`）。
- `dropout`: Dropout比率，用于防止过拟合。
- `forward_expansion`: 前馈网络（Feed-Forward Network, FFN）中隐藏层的扩展因子。

#### **组件解释**
1. **多头注意力机制（`MultiHeadAttention`）**
   ```python
   self.attention = MultiHeadAttention(embed_size, heads)
   ```
   这是Transformer的核心部分，实现了多头注意力机制。它允许模型在不同的表示子空间中学习信息。`MultiHeadAttention` 的具体实现没有在这段代码中给出，但通常它会将输入分为多个“头”，分别计算注意力权重，然后将结果拼接起来。

2. **层归一化（`LayerNorm`）**
   ```python
   self.norm1 = nn.LayerNorm(embed_size)
   self.norm2 = nn.LayerNorm(embed_size)
   ```
   层归一化（Layer Normalization）是一种归一化方法，用于稳定训练过程并加速收敛。它对每个样本的特征进行归一化，而不是像批量归一化（Batch Normalization）那样对整个批次进行归一化。

3. **前馈网络（`Feed-Forward Network`）**
   ```python
   self.feed_forward = nn.Sequential(
       nn.Linear(embed_size, forward_expansion * embed_size),
       nn.ReLU(),
       nn.Linear(forward_expansion * embed_size, embed_size),
   )
   ```
   前馈网络是一个简单的两层全连接网络。它的作用是进一步处理多头注意力机制的输出。`forward_expansion` 参数控制隐藏层的大小，通常设置为一个较大的值（如4），表示隐藏层的维度是输入维度的4倍。

4. **Dropout**
   ```python
   self.dropout = nn.Dropout(dropout)
   ```
   Dropout 是一种正则化技术，通过随机丢弃一部分神经元的输出来防止过拟合。`dropout` 参数表示丢弃的概率。

---

### 3. **前向传播方法**
```python
def forward(self, value, key, query, mask):
    attention = self.attention(value, key, query, mask)

    x = self.dropout(self.norm1(attention + query))
    forward = self.feed_forward(x)
    out = self.dropout(self.norm2(forward + x))
    return out
```

#### **参数解释**
- `value`: 值向量，用于计算注意力权重后的加权求和。
- `key`: 键向量，用于计算注意力权重。
- `query`: 查询向量，用于计算注意力权重。
- `mask`: 掩码，用于防止某些位置的信息泄露（如在自注意力中屏蔽未来信息）。

#### **流程解释**
1. **多头注意力**
   ```python
   attention = self.attention(value, key, query, mask)
   ```
   首先，使用多头注意力机制计算注意力输出。`value`、`key` 和 `query` 是输入的三个部分，`mask` 用于控制哪些位置的信息可以被关注。

2. **残差连接与层归一化**
   ```python
   x = self.dropout(self.norm1(attention + query))
   ```
   将注意力输出与输入的 `query` 进行残差连接（`attention + query`），然后通过层归一化（`LayerNorm`）。最后，应用 Dropout 防止过拟合。

3. **前馈网络**
   ```python
   forward = self.feed_forward(x)
   ```
   将经过归一化的输出传递到前馈网络中进行进一步处理。

4. **第二次残差连接与层归一化**
   ```python
   out = self.dropout(self.norm2(forward + x))
   ```
   将前馈网络的输出与之前的输出 `x` 进行残差连接，再次通过层归一化和 Dropout。

5. **返回结果**
   ```python
   return out
   ```
   最终返回处理后的输出。

---

### 4. **总结**
这段代码实现了一个标准的Transformer编码器模块，其核心包括：
- 多头注意力机制（`MultiHeadAttention`）。
- 残差连接（Residual Connection）。
- 层归一化（`LayerNorm`）。
- 前馈网络（`Feed-Forward Network`）。
- Dropout 正则化。

这些组件共同作用，使得Transformer能够高效地处理序列数据，并在许多NLP任务中取得了优异的性能。

In [15]:
# 编码器
class Encoder(nn.Module):
    def __init__(self, src_vocab_size, embed_size, num_layers, heads, device, forward_expansion, dropout, max_length):
        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.device = device
        self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
        self.position_embedding = PositionalEncoding(embed_size, dropout, max_length)

        self.layers = nn.ModuleList(
            [
                TransformerBlock(
                    embed_size,
                    heads,
                    dropout=dropout,
                    forward_expansion=forward_expansion,
                )
                for _ in range(num_layers)
            ]
        )

        self.dropout = nn.Dropout(dropout)
    def forward(self, x, mask):
            N, seq_length = x.shape
            x = self.dropout(self.position_embedding(self.word_embedding(x)))

            for layer in self.layers:
                x = layer(x, x, x, mask)

            return x


这段代码定义了一个完整的 **Transformer 编码器（Encoder）**，它是 Transformer 架构中的一个重要组成部分。编码器的作用是将输入序列（如源语言文本）转换为上下文表示，这些表示可以被解码器（Decoder）用于生成目标序列（如目标语言文本）。以下是对代码的详细解释：

---

### 1. **类定义**
```python
class Encoder(nn.Module):
```
`Encoder` 是一个继承自 PyTorch 的 `nn.Module` 的类，用于定义 Transformer 编码器的结构。`nn.Module` 是 PyTorch 中所有神经网络模块的基类，用于定义和管理神经网络的结构。

---

### 2. **初始化方法**
```python
def __init__(self, src_vocab_size, embed_size, num_layers, heads, device, forward_expansion, dropout, max_length):
    super(Encoder, self).__init__()
    self.embed_size = embed_size
    self.device = device
    self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
    self.position_embedding = PositionalEncoding(embed_size, dropout, max_length)

    self.layers = nn.ModuleList(
        [
            TransformerBlock(
                embed_size,
                heads,
                dropout=dropout,
                forward_expansion=forward_expansion,
            )
            for _ in range(num_layers)
        ]
    )

    self.dropout = nn.Dropout(dropout)
```

#### **参数解释**
- `src_vocab_size`: 源语言词汇表的大小，即输入序列中可能的标记（token）数量。
- `embed_size`: 嵌入向量的维度，表示每个词或标记的特征维度。
- `num_layers`: 编码器中 Transformer 块（`TransformerBlock`）的数量。
- `heads`: 多头注意力机制中的头数。
- `device`: 运行设备（如 CPU 或 GPU）。
- `forward_expansion`: 前馈网络（FFN）中隐藏层的扩展因子。
- `dropout`: Dropout 比率，用于防止过拟合。
- `max_length`: 输入序列的最大长度，用于位置编码。

#### **组件解释**
1. **词嵌入（`word_embedding`）**
   ```python
   self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
   ```
   词嵌入层将输入的标记（token）索引映射为固定维度的嵌入向量。`src_vocab_size` 是词汇表的大小，`embed_size` 是嵌入向量的维度。

2. **位置编码（`position_embedding`）**
   ```python
   self.position_embedding = PositionalEncoding(embed_size, dropout, max_length)
   ```
   位置编码层用于为每个标记添加位置信息，使得模型能够捕捉序列中的顺序关系。`PositionalEncoding` 的具体实现没有在这段代码中给出，但通常它会根据标记的位置生成一个固定维度的向量，并将其与词嵌入相加。

3. **Transformer 块（`TransformerBlock`）**
   ```python
   self.layers = nn.ModuleList(
       [
           TransformerBlock(
               embed_size,
               heads,
               dropout=dropout,
               forward_expansion=forward_expansion,
           )
           for _ in range(num_layers)
       ]
   )
   ```
   编码器由多个 Transformer 块组成。每个 Transformer 块包含多头注意力机制和前馈网络。`num_layers` 表示 Transformer 块的数量。

4. **Dropout**
   ```python
   self.dropout = nn.Dropout(dropout)
   ```
   Dropout 是一种正则化技术，通过随机丢弃一部分神经元的输出来防止过拟合。`dropout` 参数表示丢弃的概率。

---

### 3. **前向传播方法**
```python
def forward(self, x, mask):
    N, seq_length = x.shape
    x = self.dropout(self.position_embedding(self.word_embedding(x)))

    for layer in self.layers:
        x = layer(x, x, x, mask)

    return x
```

#### **参数解释**
- `x`: 输入序列，形状为 `(N, seq_length)`，其中 `N` 是批次大小，`seq_length` 是序列长度。
- `mask`: 掩码，用于防止某些位置的信息泄露（如在自注意力中屏蔽未来信息）。

#### **流程解释**
1. **词嵌入与位置编码**
   ```python
   x = self.dropout(self.position_embedding(self.word_embedding(x)))
   ```
   - 首先，将输入序列 `x` 通过词嵌入层（`word_embedding`）得到嵌入向量。
   - 然后，将嵌入向量与位置编码（`position_embedding`）相加，为每个标记添加位置信息。
   - 最后，应用 Dropout 防止过拟合。

2. **逐层传递**
   ```python
   for layer in self.layers:
       x = layer(x, x, x, mask)
   ```
   - 输入序列 `x` 逐层传递到每个 Transformer 块中。在每个块中：
     - `value`、`key` 和 `query` 都是 `x`，因为这是自注意力机制（Self-Attention）。
     - `mask` 用于控制哪些位置的信息可以被关注。
   - 每个 Transformer 块的输出会作为下一层的输入。

3. **返回结果**
   ```python
   return x
   ```
   - 最终返回编码器的输出，形状为 `(N, seq_length, embed_size)`，表示每个位置的上下文表示。

---

### 4. **总结**
这段代码实现了一个完整的 Transformer 编码器，其主要功能包括：
1. **词嵌入与位置编码**：将输入标记转换为嵌入向量，并添加位置信息。
2. **多层 Transformer 块**：通过多头注意力机制和前馈网络逐层处理输入序列。
3. **掩码机制**：通过掩码控制注意力的范围，避免信息泄露。
4. **Dropout 正则化**：防止过拟合。

编码器的输出是一个上下文表示，可以被解码器用于生成目标序列。这种架构在机器翻译、文本生成等任务中表现出色。

In [ ]:
# Transformer 解码器
class DecoderBlock(nn.Module):
    def __init__(self, embed_size, heads, forward_expansion, dropout, device):
        super(DecoderBlock, self).__init__()
        self.attention = MultiHeadAttention(embed_size, heads)
        self.norm = nn.LayerNorm(embed_size)
        self.transformer_block = TransformerBlock(
            embed_size, heads, dropout, forward_expansion
        )

    def forward(self, x, value, key, src_mask, trg_mask):
        attention = self.attention(x, x, x, trg_mask)
        query = self.dropout(self.norm(attention + x))
        out = self.transformer_block(value, key, query, src_mask)
        return out

这段代码定义了 Transformer 解码器中的一个基本模块——`DecoderBlock`。每个 `DecoderBlock` 包含两个主要部分：**掩码多头自注意力（Masked Multi-Head Self-Attention）** 和 **Transformer 块（TransformerBlock）**。以下是对代码的详细解析：

---

### 1. **`DecoderBlock` 的结构**

#### （1）**掩码多头自注意力（`MultiHeadAttention`）**
```python
self.attention = MultiHeadAttention(embed_size, heads)
```
- **作用**：这部分实现了掩码多头自注意力机制。
- **输入**：`x`（解码器的输入序列）。
- **掩码**：`trg_mask`（目标序列的掩码，用于防止解码器看到未来的信息）。
- **输出**：经过掩码多头自注意力处理后的张量。

#### （2）**归一化（`nn.LayerNorm`）**
```python
self.norm = nn.LayerNorm(embed_size)
```
- **作用**：对注意力输出进行归一化，以稳定训练。
- **输入**：`attention + x`（注意力输出与输入的残差连接）。
- **输出**：归一化后的张量。

#### （3）**Transformer 块（`TransformerBlock`）**
```python
self.transformer_block = TransformerBlock(embed_size, heads, dropout, forward_expansion)
```
- **作用**：这部分实现了标准的 Transformer 块，包含多头注意力和前馈网络。
- **输入**：
  - `value` 和 `key`：来自编码器的输出（编码器的上下文信息）。
  - `query`：经过归一化的解码器输入。
  - `src_mask`：编码器的掩码（用于忽略填充部分）。
- **输出**：经过 Transformer 块处理后的张量。

---

### 2. **`DecoderBlock` 的前向传播（`forward` 方法）**
```python
def forward(self, x, value, key, src_mask, trg_mask):
    attention = self.attention(x, x, x, trg_mask)
    query = self.dropout(self.norm(attention + x))
    out = self.transformer_block(value, key, query, src_mask)
    return out
```

#### （1）**掩码多头自注意力**
```python
attention = self.attention(x, x, x, trg_mask)
```
- **输入**：
  - `x`：解码器的输入序列（通常是目标序列的嵌入表示）。
  - `trg_mask`：目标序列的掩码，用于防止解码器看到未来的信息。
- **作用**：计算解码器内部的自注意力，同时通过掩码避免看到未来的信息。
- **输出**：经过掩码多头自注意力处理后的张量。

#### （2）**归一化和残差连接**
```python
query = self.dropout(self.norm(attention + x))
```
- **作用**：
  - 将注意力输出与输入 `x` 进行残差连接（`attention + x`），以保留输入的信息。
  - 使用 `LayerNorm` 对残差连接的结果进行归一化。
  - 应用 `Dropout` 进行正则化。
- **输出**：归一化后的张量，作为查询（`query`）传递到下一个模块。

#### （3）**Transformer 块**
```python
out = self.transformer_block(value, key, query, src_mask)
```
- **输入**：
  - `value` 和 `key`：来自编码器的输出（编码器的上下文信息）。
  - `query`：经过归一化的解码器输入。
  - `src_mask`：编码器的掩码（用于忽略填充部分）。
- **作用**：结合编码器的上下文信息和解码器的查询，通过 Transformer 块进行进一步处理。
- **输出**：经过 Transformer 块处理后的张量。

---

### 3. **`DecoderBlock` 的作用**
`DecoderBlock` 是 Transformer 解码器的核心模块，它结合了以下两个主要功能：
1. **掩码多头自注意力**：
   - 用于处理解码器内部的序列，同时通过掩码避免看到未来的信息。
2. **编码器-解码器注意力**：
   - 通过 `TransformerBlock`，结合编码器的上下文信息（`value` 和 `key`）和解码器的查询（`query`），生成最终的输出。

---

### 4. **总结**
`DecoderBlock` 是 Transformer 解码器中的一个模块，它包含：
- **掩码多头自注意力**：处理解码器内部的序列。
- **归一化和残差连接**：稳定训练并保留输入信息。
- **Transformer 块**：结合编码器的上下文信息和解码器的查询，生成最终输出。

通过多个 `DecoderBlock` 的堆叠，解码器能够逐步生成目标序列，同时利用编码器提供的上下文信息。

In [16]:
# 解码器
class Decoder(nn.Module):
    def __init__(self, trg_vocab_size, embed_size, num_layers, heads, forward_expansion, dropout, device, max_length):
        super(Decoder, self).__init__()
        self.device = device
        self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)
        self.position_embedding = PositionalEncoding(embed_size, dropout, max_length)

        self.layers = nn.ModuleList(
            [
                DecoderBlock(embed_size, heads, forward_expansion, dropout, device)
                for _ in range(num_layers)
            ]
        )

        self.fc_out = nn.Linear(embed_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_out, src_mask, trg_mask):
        N, seq_length = x.shape
        x = self.dropout(self.position_embedding(self.word_embedding(x)))

        for layer in self.layers:
            x = layer(x, enc_out, enc_out, src_mask, trg_mask)

        out = self.fc_out(x)
        return out

In [17]:
# Transformer 模型
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx, embed_size=256, num_layers=6, forward_expansion=4, heads=8, dropout=0.1, max_length=100, device="cuda"):
        super(Transformer, self).__init__()
        self.encoder = Encoder(src_vocab_size, embed_size, num_layers, heads, device, forward_expansion, dropout, max_length)
        self.decoder = Decoder(trg_vocab_size, embed_size, num_layers, heads, forward_expansion, dropout, device, max_length)
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device

    def make_src_mask(self, src):
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        return src_mask.to(self.device)

    def make_trg_mask(self, trg):
        N, trg_len = trg.shape
        trg_mask = torch.tril(torch.ones((trg_len, trg_len))).expand(
            N, 1, trg_len, trg_len
        )
        return trg_mask.to(self.device)

    def forward(self, src, trg):
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        enc_src = self.encoder(src, src_mask)
        out = self.decoder(trg, enc_src, src_mask, trg_mask)
        return out

# 超参数和模型实例化
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Transformer(src_vocab_size=10000, trg_vocab_size=10000, src_pad_idx=0, trg_pad_idx=0, embed_size=256, num_layers=6, forward_expansion=4, heads=8, dropout=0.1, max_length=100, device=device)

# 模拟数据
src = torch.tensor([[1, 5, 6, 2, 3, 0], [4, 3, 2, 9, 0, 0]], dtype=torch.long).to(device)
trg = torch.tensor([[1, 7, 4, 3, 5, 2], [1, 5, 6, 2, 0, 0]], dtype=torch.long).to(device)

out = model(src, trg[:, :-1])
print(out.shape)  # 输出形状应为 (batch_size, trg_len - 1, trg_vocab_size)

NameError: name 'PositionalEncoding' is not defined


### 代码说明：
1. **位置编码（`PositionalEncoding`）**：为输入序列添加位置信息。
2. **多头注意力（`MultiHeadAttention`）**：实现多头注意力机制。
3. **Transformer 块（`TransformerBlock`）**：包含多头注意力和前馈网络。
4. **编码器（`Encoder`）**：由多个 Transformer 块组成。
5. **解码器（`Decoder`）**：由多个解码器块组成，每个块包含多头注意力和 Transformer 块。
6. **Transformer 模型（`Transformer`）**：整合编码器和解码器，并生成掩码。

### 注意事项：
- 代码中使用了 `torch.tril` 来生成解码器的掩码，防止解码器看到未来的信息。
- 输入数据需要进行适当的预处理，包括词汇表的构建和填充标记的处理。
- 该代码是一个基础实现，可以根据具体任务进行调整和优化。

这个代码实现了一个完整的 Transformer 模型，包括编码器（Encoder）和解码器（Decoder），用于序列到序列的任务（如机器翻译）。以下是对模型结构和参数量的详细解析。

---

### 1. **模型结构解析**

#### （1）**位置编码（`PositionalEncoding`）**
- **作用**：为输入的嵌入向量添加位置信息，使得 Transformer 能够感知序列中的位置。
- **结构**：
  - 使用正弦和余弦函数生成位置编码矩阵。
  - 位置编码的形状为 `(max_len, d_model)`。
  - 通过 `torch.sin` 和 `torch.cos` 计算每个位置的编码。
  - 使用 `nn.Dropout` 对位置编码后的嵌入向量进行正则化。
- **参数量**：无学习参数（位置编码是固定的，不参与训练）。

#### （2）**多头注意力（`MultiHeadAttention`）**
- **作用**：将输入分割成多个“头”，每个头学习不同的特征，然后将结果合并。
- **结构**：
  - `self.values`、`self.keys`、`self.queries`：三个线性层，分别将输入转换为值（Values）、键（Keys）和查询（Queries）。
  - `self.fc_out`：线性层，将多头的输出拼接后转换回原始嵌入维度。
- **参数量**：
  - 每个线性层的参数量为 \(d_{model} \times d_{model}\)。
  - 总参数量：\(4 \times d_{model}^2\)。

#### （3）**Transformer 块（`TransformerBlock`）**
- **作用**：包含多头注意力和前馈网络（Feed-Forward Network）。
- **结构**：
  - **多头注意力**：通过 `MultiHeadAttention` 实现。
  - **LayerNorm**：归一化层，用于稳定训练。
  - **前馈网络**：包含两个线性层，中间有 ReLU 激活函数。
  - **Dropout**：用于正则化。
- **参数量**：
  - 多头注意力：\(4 \times d_{model}^2\)。
  - 前馈网络：
    - 第一个线性层：\(d_{model} \times (d_{model} \times forward\_expansion)\)。
    - 第二个线性层：\((d_{model} \times forward\_expansion) \times d_{model}\)。
  - 总参数量：\(4 \times d_{model}^2 + 2 \times d_{model}^2 \times forward\_expansion\)。

#### （4）**编码器（`Encoder`）**
- **作用**：对输入序列进行编码。
- **结构**：
  - **词嵌入**：`nn.Embedding`，将输入的单词索引转换为嵌入向量。
  - **位置编码**：通过 `PositionalEncoding` 添加位置信息。
  - **多层 Transformer 块**：包含多个 `TransformerBlock`。
- **参数量**：
  - 词嵌入：\(src\_vocab\_size \times d_{model}\)。
  - 每个 Transformer 块：\(4 \times d_{model}^2 + 2 \times d_{model}^2 \times forward\_expansion\)。
  - 总参数量：\(src\_vocab\_size \times d_{model} + num\_layers \times (4 \times d_{model}^2 + 2 \times d_{model}^2 \times forward\_expansion)\)。

#### （5）**解码器（`Decoder`）**
- **作用**：根据编码器的输出生成目标序列。
- **结构**：
  - **词嵌入**：`nn.Embedding`，将目标单词索引转换为嵌入向量。
  - **位置编码**：通过 `PositionalEncoding` 添加位置信息。
  - **多层 Decoder 块**：包含多个 `DecoderBlock`，每个块包含多头注意力和 Transformer 块。
  - **输出层**：`nn.Linear`，将解码器的输出转换为目标词汇表大小。
- **参数量**：
  - 词嵌入：\(trg\_vocab\_size \times d_{model}\)。
  - 每个 Decoder 块：\(4 \times d_{model}^2 + 2 \times d_{model}^2 \times forward\_expansion\)。
  - 输出层：\(d_{model} \times trg\_vocab\_size\)。
  - 总参数量：\(trg\_vocab\_size \times d_{model} + num\_layers \times (4 \times d_{model}^2 + 2 \times d_{model}^2 \times forward\_expansion) + d_{model} \times trg\_vocab\_size\)。

#### （6）**Transformer 模型（`Transformer`）**
- **作用**：整合编码器和解码器，完成序列到序列的任务。
- **结构**：
  - 编码器：`Encoder`。
  - 解码器：`Decoder`。
  - 掩码生成：`make_src_mask` 和 `make_trg_mask`，用于生成编码器和解码器的掩码。
- **参数量**：
  - 编码器和解码器的参数量之和。

---

### 2. **参数量计算**
假设：
- `src_vocab_size = 10000`
- `trg_vocab_size = 10000`
- `d_model = 256`
- `num_layers = 6`
- `forward_expansion = 4`
- `heads = 8`

#### （1）**位置编码**
- 无学习参数。

#### （2）**多头注意力**
- 参数量：\(4 \times d_{model}^2 = 4 \times 256^2 = 262144\)。

#### （3）**Transformer 块**
- 前馈网络：
  - 第一个线性层：\(d_{model} \times (d_{model} \times forward\_expansion) = 256 \times (256 \times 4) = 262144\)。
  - 第二个线性层：\((d_{model} \times forward\_expansion) \times d_{model} = (256 \times 4) \times 256 = 262144\)。
- 总参数量：\(4 \times 256^2 + 2 \times 262144 = 786432\)。

#### （4）**编码器**
- 词嵌入：\(src\_vocab\_size \times d_{model} = 10000 \times 256 = 2560000\)。
- 每个 Transformer 块：\(786432\)。
- 总参数量：\(2560000 + 6 \times 786432 = 7218592\)。

#### （5）**解码器**
- 词嵌入：\(trg\_vocab\_size \times d_{model} = 10000 \times 256 = 2560000\)。
- 每个 Decoder 块：\(786432\)。
- 输出层：\(d_{model} \times trg\_vocab\_size = 256 \times 10000 = 2560000\)。
- 总参数量：\(2560000 + 6 \times 786432 + 2560000 = 7218592\)。

#### （6）**Transformer 模型**
- 总参数量：\(7218592 + 7218592 = 14437184\)。

---

### 3. **模型结构总结**
这个 Transformer 模型包含以下主要部分：
1. **编码器（Encoder）**：
   - 词嵌入和位置编码。
   - 多层 Transformer 块，每层包含多头注意力和前馈网络。
2. **解码器（Decoder）**：
   - 词嵌入和位置编码。
   - 多层 Decoder 块，每块包含多头注意力和 Transformer 块。
   - 输出层将解码器的输出转换为目标词汇表大小。
3. **掩码生成**：
   - 编码器掩码：忽略填充部分。
   - 解码器掩码：忽略未来信息和填充部分。

### 4. **参数量总结**
- **编码器参数量**：约 721 万。
- **解码器参数量**：约 721 万。
- **总参数量**：约 1443 万。

这个模型是一个典型的 Transformer 架构，适用于机器翻译、文本生成等任务。通过多头注意力机制和前馈网络，模型能够有效地捕捉序列中的长距离依赖关系。